https://www.flickr.com/

# Install necessary libraries

In [ ]:
%pip install tqdm

In [ ]:
!apt-get update
!apt-get install -y wget
%pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,787 kB]
Get:14 http

In [ ]:
from bs4 import BeautifulSoup  # For parsing HTML content
from urllib.parse import urljoin, urlparse  # For handling URLs
import urllib.request  # For making HTTP requests
import time  # For handling time-related operations
import os  # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm  # For displaying progress bars (visualize progress)
import concurrent.futures  # For multi-threading
import json  # For writing to a text file
from PIL import Image  # For handling images

# Crawl image links from website (Brute force)

In [ ]:
URL = "https://www.flickr.com/search/?text="
search_term = 'cat'

In [ ]:
def get_url_images(driver, max_images):
    """
    Crawl the urls of images by term

    Parameters:
    term (str): The name of animal, plant, scenery, furniture

    Returns:
    urls (list): List of urls of images
    """
    urls = [] # Store links of all images we want
    more_content_available = True # Checking variable for manage lazy loading page

    # Crawl url images until the number of link = max img we want or the page don't have any image
    while len(urls) < max_images and more_content_available:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        img_tags = soup.find_all("img")

        # Extract url image from HTML source
        for img in img_tags:
            if len(urls) >= max_images:
                break
            if 'src' in img.attrs:
                href = img.attrs['src']
                img_path = urljoin(url, href)
                img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                    continue
                urls.append(img_path)


        # Click load more button or scroll page for more image
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
            )
            load_more_button.click()
            time.sleep(2) # Wait for generating content
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # Wait for generating content

            # Check number of new generating image
            new_soup = BeautifulSoup(driver.page_source, "html.parser")
            new_img_tags = new_soup.find_all("img", loading_="lazy")
            if len(new_img_tags) == len(img_tags):
                more_content_available = False
            img_tags = new_img_tags

    return urls

In [ ]:
url = URL + search_term
max_images = 200

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.get(url)

urls = get_url_images(driver, max_images)
print(f"Number of images retrieved: {len(urls)}")
driver.quit()

Number of images retrieved: 200


In [ ]:
urls

['https://live.staticflickr.com/7697/17026317426_bb3acf19fb_b.jpg',
 'https://live.staticflickr.com/7302/11991979753_eac1d33dd8_b.jpg',
 'https://live.staticflickr.com/3845/14284324148_360ae173b8_b.jpg',
 'https://live.staticflickr.com/4728/39256672581_c48569e3a2_b.jpg',
 'https://live.staticflickr.com/7073/7190755946_ea97e85765_b.jpg',
 'https://live.staticflickr.com/3946/14934292534_32b0302f2d_b.jpg',
 'https://live.staticflickr.com/5036/5881933297_7974eaff82_b.jpg',
 'https://live.staticflickr.com/2602/3977203168_b9d02a0233_b.jpg',
 'https://live.staticflickr.com/3940/15504684310_f555c88915_b.jpg',
 'https://live.staticflickr.com/7313/9775005856_9b5e0ebe16_b.jpg',
 'https://live.staticflickr.com/1701/24811748270_3102fc52fb_b.jpg',
 'https://live.staticflickr.com/5170/5372754294_db6acaa1e5_b.jpg',
 'https://live.staticflickr.com/4733/27257168879_464200ea90_b.jpg',
 'https://live.staticflickr.com/8208/8216315457_28762c496d_b.jpg',
 'https://live.staticflickr.com/6100/6303228181_59371c

# Crawl image links from website (More reusable and efficient)

In [ ]:
class UrlScraper:
    # Constructor
    def __init__(self, url_template, max_images=50, max_workers=4):
        self.url_template = url_template # Link crawl
        self.max_images = max_images # Max images
        self.max_workers = max_workers # Thread
        self.setup_environment() # Call for set up environment

    # Set up environment for selenium
    def setup_environment(self):
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Crawl the urls of images by term

        Parameters:
        term (str): The name of animal, plant, scenery, furniture

        Returns:
        urls (list): List of urls of images
        """

        # Initialize Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Start crawl urls of image like brute force - the same mechanism with this but add some feature
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}") # Set up for visualize progress

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            img_tags = soup.find_all("img")

            for img in img_tags:
                if len(urls) >= self.max_images:
                    break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(url, href)
                    img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)

            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
                )
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                new_soup = BeautifulSoup(driver.page_source, "html.parser")
                new_img_tags = new_soup.find_all("img", loading_="lazy")
                if len(new_img_tags) == len(img_tags):
                    more_content_available = False
                img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Call get_url_images method to get all urls of any object in categories\

        Parameter:
        categories (dictionary): the dict of all object we need to collect image with format categories{"name_object": [value1, value2, ...]}

        Returns:
        all_urls (dictionary): Dictionary of urls of images
        """
        all_urls = {category: {} for category in categories}

        # Handle multi-threading for efficent installation
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                              for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(future_to_term), desc="Overall Progress"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nNumber of images retrieved for {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} generated an exception: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Save the data to a JSON file.

        Parameters:
        data (dict): The data to be saved.
        filename (str): The name of the JSON file.

        Returns:
        None
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename}")

In [ ]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse", "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel", "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat", "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake", "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden", "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]
}

In [ ]:
# categories = {
#     "animal": ["horse", "pig", "Alligator", "bird"],
#     "plant": ["apple", "carrot", "flower"],
#     "furniture": ["table", "Piano", "Bookcase", "Umbrella", "book"],
#     "scenery": ["fireworks", "sky", "cave", "cloud"]
# }

In [ ]:
urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)

Fetching images for cows:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Dog:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for Elephant:   0%|          | 0/20 [00:00<?, ?it/s]



Fetching images for Monkey:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Dog:   5%|▌         | 1/20 [00:01<00:25,  1.32s/it]

Fetching images for Dog:  10%|█         | 2/20 [00:01<00:11,  1.61it/s]

Fetching images for Dog:  15%|█▌        | 3/20 [00:01<00:06,  2.53it/s]




Fetching images for Cat:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Dog:  20%|██        | 4/20 [00:01<00:04,  3.51it/s]

Fetching images for cows:  10%|█         | 2/20 [00:02<00:19,  1.08s/it]



Fetching images for Monkey:   5%|▌         | 1/20 [00:01<00:33,  1.78s/it]


Fetching images for Elephant:   5%|▌         | 1/20 [00:01<00:36,  1.94s/it]




Overall Progress:   1%|          | 1/134 [00:34<1:15:31, 34.07s/it]


Number of images retrieved for Dog: 20


Overall Progress:   2%|▏         | 3/134 [00:35<17:39,  8.09s/it]


Number of images retrieved for Monkey: 20

Number of images retrieved for cows: 20


Fetching images for Cat: 100%|██████████| 20/20 [00:18<00:00,  1.08it/s]


...


Number of images retrieved for plateau: 20


Overall Progress:  99%|█████████▊| 132/134 [12:13<00:08,  4.02s/it]


Number of images retrieved for valley: 20

Number of images retrieved for volcano: 20


Overall Progress: 100%|██████████| 134/134 [12:22<00:00,  5.54s/it]


Number of images retrieved for waterfall: 20


In [ ]:
scraper.save_to_file(image_urls, 'image_urls.json')

Data saved to image_urls.json


# Downloading Images from JSON file

In [ ]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # file containing URLs of images in JSON format
        self.download_dir = download_dir  # Folder name for storing images
        self.max_workers = max_workers  # Number of threads
        self.delay = delay  # Polite delay: when we send request too much to the server for downloading images without polite delay, it will crash or prevent your IP
        self.filename = set()  # To store filename directories
        self.setup_directory()  # Set up the folder structure

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Read the JSON file and return the data.

        Returns:
        data (dict): The data read from the JSON file.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Check if the URL is valid.

        Parameters:
        url (str): The URL to be checked.

        Returns:
        bool: True if the URL is valid, False otherwise.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Download the image from the given URL.

        Parameters:
        url (str): The URL of the image to be downloaded.
        category (str): The category of the image.
        term (str): The term or keyword associated with the image.
        pbar (tqdm): The progress bar object.

        Returns:
        str: A message indicating the status of the download.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename)  # Record the filename directory

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Download images from the JSON file.

        Returns:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Polite delay

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Export the filename directories to a text file.

        Returns:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")


In [ ]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()

Downloaded: https://live.staticflickr.com/6174/6138018661_eb5f9cc445_b.jpg
Downloaded: https://live.staticflickr.com/3369/3340532547_d552579b4a_b.jpg
Downloaded: https://live.staticflickr.com/7582/16143796666_6fdf2e7f52_b.jpg
Downloaded: https://live.staticflickr.com/7377/27844318470_bd67ec691a_b.jpg
Downloaded: https://live.staticflickr.com/581/31472011111_88dd615576_b.jpg
Downloaded: https://live.staticflickr.com/4147/4977427214_d2c3f22c3c_b.jpg
Downloaded: https://live.staticflickr.com/4324/35920929402_257566d5a7_b.jpg
Downloaded: https://live.staticflickr.com/8323/8446750937_bd821ab062_b.jpg
Downloaded: https://live.staticflickr.com/4569/38171716511_a755a8b569_b.jpg
Downloaded: https://live.staticflickr.com/5667/22836756693_e45173451d_b.jpg
Downloaded: https://live.staticflickr.com/4074/4883477375_e4e2c8b2cb_b.jpg
Downloaded: https://live.staticflickr.com/8507/8523369234_a0215f3f82_b.jpg
Downloaded: https://live.staticflickr.com/5605/14940155724_255701a815_b.jpg
Downloaded: https:/

Downloaded: https://live.staticflickr.com/3409/3310999623_b091a9181c_b.jpg
Downloaded: https://live.staticflickr.com/3083/2865338391_a419230b3e_b.jpg
Downloaded: https://live.staticflickr.com/2674/3675748589_b8dc02d6d4_b.jpg
Downloaded: https://live.staticflickr.com/44/126328216_71388871e1_b.jpg
Downloaded: https://live.staticflickr.com/5178/5459216987_c9986a4b68_b.jpg
Downloaded: https://live.staticflickr.com/34/102658557_042c4738d8_b.jpg
Downloaded: https://live.staticflickr.com/52/136297507_7d6bad75ca_b.jpg
Downloaded: https://live.staticflickr.com/228/510416672_1f1dc3135e_b.jpg
Downloaded: https://live.staticflickr.com/4579/38557126836_f784078e8b_b.jpg
Downloaded: https://live.staticflickr.com/3465/3989693058_0a21d794b1_b.jpg
Downloaded: https://live.staticflickr.com/5343/30618799306_f778aeae12_b.jpg
Downloaded: https://live.staticflickr.com/7227/7212011420_013f325a69_b.jpg
Downloaded: https://live.staticflickr.com/8502/8324866332_cb7f5cd6f4_b.jpg
Downloaded: https://live.staticfli

In [ ]:
downloader.export_filename()

## Zip folder dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!ls

Dataset  drive	filename.txt  image_urls.json  sample_data


In [ ]:
!zip -r /content/drive/MyDrive/Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/furniture/ (stored 0%)
  adding: Dataset/furniture/cabinet/ (stored 0%)
  adding: Dataset/furniture/cabinet/13211984934_c2ba060f18_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211485103_25eb3847cb_b.jpg (deflated 1%)
  adding: Dataset/furniture/cabinet/13211481643_d3b26a089b_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211621195_21243d1d64_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/4407189112_d291cd76f0_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211788443_f0489ddf8a_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211889575_de0406d4ea_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13212493915_75d72d6418_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211936983_a8d62abc85_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13212908614_ff59f5a567_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/112649353_02ed95fff0_b.jpg (deflated 0%)
  adding: Dat

### Processing Dataset

In [ ]:
def check_and_preprocess_images(image_dir):
    """
    Check and preprocess images in the specified directory.

    Parameters:
    image_dir (str): The directory containing the images to be checked and preprocessed.

    Returns:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

In [ ]:
check_and_preprocess_images('Dataset')

Converted Dataset/plant/Bean/8529951413_c315df19e5_b.jpg to RGB
Converted Dataset/animal/Cheetah/4838391253_bed2279fdf_b.jpg to RGB
Converted Dataset/animal/Cheetah/4838389045_d5fc08ca2b_b.jpg to RGB
Converted Dataset/animal/Whale/10004272976_0ea81a97b4_b.jpg to RGB
Converted Dataset/scenery/volcano/49629211101_d46f735055_b.jpg to RGB
Converted Dataset/scenery/desert/48115750033_ed68297f86_b.jpg to RGB
Converted Dataset/scenery/desert/48876880212_34d35f2905_b.jpg to RGB
Converted Dataset/scenery/desert/49510379992_948ff98ebd_b.jpg to RGB
Converted Dataset/scenery/desert/52712003999_cba6904a86_b.jpg to RGB
Converted Dataset/scenery/desert/49011833281_72063e1544_b.jpg to RGB


In [ ]:
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/furniture/ (stored 0%)
  adding: Dataset/furniture/cabinet/ (stored 0%)
  adding: Dataset/furniture/cabinet/13211984934_c2ba060f18_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211485103_25eb3847cb_b.jpg (deflated 1%)
  adding: Dataset/furniture/cabinet/13211481643_d3b26a089b_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211621195_21243d1d64_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/4407189112_d291cd76f0_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211788443_f0489ddf8a_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211889575_de0406d4ea_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13212493915_75d72d6418_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13211936983_a8d62abc85_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/13212908614_ff59f5a567_b.jpg (deflated 0%)
  adding: Dataset/furniture/cabinet/112649353_02ed95fff0_b.jpg (deflated 0%)
  adding: Dat

In [ ]:
!cp filename.txt /content/drive/MyDrive/filename.txt